In [1]:
#pip install --upgrade tomotopy
#pip install --upgrade pandas
#pip install reshape
#pip install gensim

import re
import gensim
import numpy as np
import tomotopy as tp
import tomotopy.coherence
from kiwipiepy import Kiwi, Option
from pyvis.network import Network

In [2]:
kiwi = Kiwi()

In [3]:
Kiwi(num_workers=-0, model_path='./', options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH)

In [4]:
try:
    corpus = tp.utils.Corpus.load('k.cps')
except IOError:
    kiwi = Kiwi()
    kiwi.add_user_word('전달력', tag='NNG')
    kiwi.add_user_word('영상미', tag='NNG')
    kiwi.add_user_word('영상기법', tag='NNG')
    kiwi.add_user_word('영상구도', tag='NNG')
    kiwi.add_user_word('배경음악', tag='NNG')
    kiwi.add_user_word('구독자수', tag='NNG')   
    kiwi.add_user_word('레고프렌즈', tag='NNG')
    kiwi.add_user_word('딱히', tag='MAG')
    kiwi.add_user_word('교육', tag='NNP')
    kiwi.add_user_word('방송', tag='NNP')
    kiwi.add_user_word('브이로그', tag='NNP')
    kiwi.add_user_word('생일', tag='NNP')
    kiwi.add_user_word('날', tag='NNP')
    kiwi.add_user_word('전반적', tag='NNP')
    kiwi.add_user_word('조회수', tag='NNP')
    kiwi.add_user_word('매니아층', tag='NNP')
    kiwi.add_user_word('생방송', tag='NNP')
    kiwi.add_user_word('먹방', tag='NNP')
    kiwi.add_user_word('배경음악', tag='NNP')
    kiwi.add_user_word('맛집', tag='NNP')
    kiwi.add_user_word('재건축', tag='NNP')
    kiwi.add_user_word('언박싱', tag='NNP')    
    kiwi.prepare()
    pat_tag = re.compile('NN[GP]|V[VA]|MAG|MM') 

In [5]:
    def tokenizer(raw, user_data):
        res, _ = user_data()[0]
        for w, tag, start, l in res:
            if not pat_tag.match(tag) or len(w) <= 1: continue
            yield w + ('다' if tag.startswith('V') else ''), start, l

In [6]:
    corpus = tp.utils.Corpus(
        tokenizer=tokenizer
    )
    
    corpus.process((line, kiwi.async_analyze(line)) for line in open('ctm_data_final_210317.txt', encoding='UTF-8'))
    corpus.save('data_210317_2.cps')

case1. tw기본값 (tw=tp.TermWeight.ONE)으로 세팅했을 때!

In [15]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

#아래는 코드 (#지우기)
#mdl = tp.CTModel(tw=tp.TermWeight.ONE, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
#mdl.train(0)

case2. tw에 PMI로 가중치 부여했을 때

In [ ]:
# 최소 1개 이상 문헌에 등장하고, 전체 출현빈도는 2 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 20개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

#아래는 코드 (#지우기)
#mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=1, min_cf=2, rm_top=0, k=20, corpus=corpus)
#mdl.train(0)

case3. tw에 PMI로 가중치 부여하고, min_df=1, min_cf=1로 설정 (개발자분 피드백)

In [8]:
# 최소 0개 이상 문헌에 등장하고, 전체 출현빈도는 0 이상인 단어만 사용
# 그리고 상위 10개 고빈도 단어는 분석에서 제외하고싶으면 rm_top=10을 추가, 안하고 싶으면 0
# 주제 개수는 10개

#tw=용어 가중치(tp.TermWeight.ONE은 기본값으로 모든 용어의 가중치를 동일하게 간주)

mdl = tp.CTModel(tw=tp.TermWeight.PMI, min_df=0, min_cf=0, rm_top=0, k=10, corpus=corpus)
mdl.train(0)

In [9]:
#num_beta_sample은 만 개 이상이면 1~5, 수 천개라면 최소 10, 수 백개에 불과하면 20이상으로 키우기
mdl.num_beta_sample = 10
print('Num docs:{}, Total Words:{}'.format(
    len(mdl.docs), mdl.num_words
))

Num docs:147, Total Words:465


In [10]:
#깁스샘플링
for i in range(0, 1000, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)

Iteration: 0000, LL per word: -27.45
Iteration: 0020, LL per word: -25.43
Iteration: 0040, LL per word: -26.71
Iteration: 0060, LL per word: -28.57
Iteration: 0080, LL per word: -28.24
Iteration: 0100, LL per word: -29.28
Iteration: 0120, LL per word: -28.49
Iteration: 0140, LL per word: -29.21
Iteration: 0160, LL per word: -29.7
Iteration: 0180, LL per word: -28.78
Iteration: 0200, LL per word: -29.08
Iteration: 0220, LL per word: -29.52
Iteration: 0240, LL per word: -29.66
Iteration: 0260, LL per word: -28.93
Iteration: 0280, LL per word: -28.85
Iteration: 0300, LL per word: -28.46
Iteration: 0320, LL per word: -28.93
Iteration: 0340, LL per word: -28.56
Iteration: 0360, LL per word: -29.3
Iteration: 0380, LL per word: -28.95
Iteration: 0400, LL per word: -28.96
Iteration: 0420, LL per word: -29.69
Iteration: 0440, LL per word: -29.84
Iteration: 0460, LL per word: -29.42
Iteration: 0480, LL per word: -29.12
Iteration: 0500, LL per word: -28.43
Iteration: 0520, LL per word: -28.5
Iter

In [11]:
print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))

Iteration: 0980, LL per word: -28.06


In [12]:
mdl.summary()

#Words: CTM 분석 후 추출된 문서들 전체의 단어의 개수 (문서 당 같은 단어가 2개 있을 수도 있고 중복 가능)
#Total Vocabs: 작은 빈도의 단어들을 제거한 뒤 남은 토픽모델링 후 어휘의 개수
#Used_Vocabs: 모델에 실제로 사용된 어휘만을 포함하는 dictionary 타입의 어휘 사전
#prior_cov : 토픽의 사전분포인 로지스틱 정규분포의 공분산 행렬
#prior_mean : 토픽의 사전분포인 로지스틱 정규분포의 평균 벡터
#k : 토픽의 개수, 1 ~ 32767 범위의 정수.
#alpha는 CTM 모형의 모수 
#eta는 모수 
#num.iterations는 반복 횟수로 z를 몇 번 추출하느냐


<Basic Info>
| CTModel (current version: 0.10.2)
| 147 docs, 465 words
| Total Vocabs: 273, Used Vocabs: 273
| Entropy of words: -5.33406
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 1000, Burn-in steps: 0
| Optimization Interval: 2
| Log-likelihood per word: -28.06273
|
<Initial Parameters>
| tw: TermWeight.PMI
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| smoothing_alpha: 0.1 (small smoothing value for preventing topic counts to be zero)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 547930673 (random seed)
| trained in version 0.10.2
|
<Parameters>
| prior_mean (Prior mean of Logit-normal for the per-document topic distributions)
|  [-4.1397047 -4.2771854 -4.144798  -2.9072309 -4.4188123 -3.843314
|   -4.112764  -4.033484  -2.6398475 -2.3532903]
| prior_cov (Prior covariance of Logit-no

In [13]:
mdl.save('trained_ctm_model.bin')

In [14]:
for preset in ('u_mass', 'c_uci', 'c_npmi', 'c_v'):
    coh = tp.coherence.Coherence(mdl, coherence=preset)
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
    print('==== Coherence : {} ===='.format(preset))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()

==== Coherence : u_mass ====
Average: -22.695269587683853 
Per Topic: [-19.421244469839415, -25.199343956493824, -22.29227354169433, -25.30077391629495, -25.30663981731466, -19.444854986330444, -18.15599961061216, -24.678924100531002, -20.709848867631784, -26.442792610095985]

==== Coherence : c_uci ====
Average: -15.372814631251398 
Per Topic: [-13.409686511932275, -16.14768570650404, -14.929257712545175, -17.801549411497405, -16.746832157186507, -12.985294904445828, -12.317418477567301, -16.55002952626775, -14.854377833359669, -17.986014071208043]

==== Coherence : c_npmi ====
Average: -0.4683005924503026 
Per Topic: [-0.35099741298801457, -0.5155628128197335, -0.4283757736107375, -0.6006074424353394, -0.5583287931603876, -0.34703604791350234, -0.26685749071697545, -0.54890795004523, -0.4391583439557247, -0.6271738568573811]

==== Coherence : c_v ====
Average: 0.9185593688488007 
Per Topic: [0.8915929615497589, 0.9541544616222382, 0.9357196390628815, 0.9196041285991668, 0.93589260578

In [17]:
import itertools

In [33]:
for seg, cm, im in itertools.product(tp.coherence.Segmentation, tp.coherence.ConfirmMeasure, tp.coherence.IndirectMeasure):
    coh = tp.coherence.Coherence(mdl, coherence=(tp.coherence.ProbEstimation.DOCUMENT, seg, cm, im))
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
    print('==== Coherence : {}, {}, {} ===='.format(repr(seg), repr(cm), repr(im)))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()

==== Coherence : <Segmentation.ONE_ONE: 1>, <ConfirmMeasure.DIFFERENCE: 1>, <IndirectMeasure.NONE: 0> ====
Average: 0.0648883849772805 
Per Topic: [0.11020408162494019, 0.07195767194605714, 0.0767573696087279, 0.02562988157796251, 0.04052910052218984, 0.06748677248228047, 0.1564436885760537, 0.03631897202800393, 0.05444822372956853, 0.009108087677020878]

==== Coherence : <Segmentation.ONE_ONE: 1>, <ConfirmMeasure.DIFFERENCE: 1>, <IndirectMeasure.COSINE: 1> ====
Average: 0.08934091739102612 
Per Topic: [0.11152251205510563, 0.011295983733402359, 0.10901049705098743, 0.122258392152273, 0.04723568952694121, 0.09798650960955355, 0.1641538253319191, 0.1012679673808937, 0.05954391576815397, 0.0691338813010311]

==== Coherence : <Segmentation.ONE_ONE: 1>, <ConfirmMeasure.DIFFERENCE: 1>, <IndirectMeasure.DICE: 2> ====
Average: 149131.73289229168 
Per Topic: [-4.3399435420831045, 1491305.331062754, 4.124833387798733, 2.26373837656445, 0.5873201436466641, 2.4838722176022, 0.024737685877415868, 

==== Coherence : <Segmentation.ONE_SET: 5>, <ConfirmMeasure.PMI: 5>, <IndirectMeasure.DICE: 2> ====
Average: 0.9091014659404755 
Per Topic: [0.9091062843799591, 0.9090942203998565, 0.9090932488441468, 0.9090989291667938, 0.9090974628925323, 0.9090966045856476, 0.909101402759552, 0.9091117680072784, 0.9091202974319458, 0.9090944409370423]

==== Coherence : <Segmentation.ONE_SET: 5>, <ConfirmMeasure.PMI: 5>, <IndirectMeasure.JACCARD: 3> ====
Average: 10.014815587997436 
Per Topic: [10.021323871612548, 10.004312419891358, 10.003182220458985, 10.011092567443848, 10.008977127075195, 10.007648468017578, 10.01440896987915, 10.031373310089112, 10.041115856170654, 10.004721069335938]

==== Coherence : <Segmentation.ONE_SET: 5>, <ConfirmMeasure.NPMI: 6>, <IndirectMeasure.NONE: 0> ====
Average: 0.0 
Per Topic: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

==== Coherence : <Segmentation.ONE_SET: 5>, <ConfirmMeasure.NPMI: 6>, <IndirectMeasure.COSINE: 1> ====
Average: 0.9167887419462204 
Per T

In [19]:
import matplotlib.pyplot as plt

In [50]:
import gensim

In [21]:
mdl.used_vocabs   

#Used Vocabs 의미 >> 78개
#242 words >> k(20)에 속한 총 단어의 수 (84개가 중복되어서 들어감)  == 총 단어의 빈도 수

['영상', '브이로그', '일상', '게임', '편집', '재미있다', '좋아하다', '만들다', '사람', '정보', '주제', '여행', '자유', '소통', '공유', '많이', '모습', '강아지', '대하다', '가장', '활동', '시청자', '도움', '관심', '패션', '생활', '콘텐츠', '업로드', '라이브', '보이다', '추억', '운동', '취미', '교육', '귀엽다', '분야', '관련', '처음', '방송', '실시간', '제작', '즐거움', '음악', '내용', '먹방', '촬영', '자연', '영상미', '고양이', '수업', '아이', '직장', '반려', '즐기다', '함께', '노력', '직접', '연주', '노래', '지식', '전달', '기존', '리뷰', '위하다', '사회', '경험', '기록', '어리다', '공감', '나오다', '가족', '요리', '배경음악', '신경', '요소', '제품', '비교', '올리다', '사용', '다이어트', '새롭다', '메뉴', '딱히', '구독자수', '얽매이다', '저장', '건강', '차량', '시승', '전달력', '메추리', '실력', '다양', '접근성', '용이', '남기다', '온라인', '헬스장', '다른', '동료', '동호회', '구입', '골프', '연습', '방법', '환경', '정성', '들어가다', '자녀', '역사', '컨셉', '생각', '그저', '그대로', '산책', '해외', '소개', '패션쇼', '유명한', '밴드', '신규', '접하다', '겨울왕국', '라이', '특징', '후원', '채팅', '식물', '키우다', '실무', '동기', '동창', '리액션', '너무', '공감대', '다시', '지금', '학교', '과제', '시연', '스피드', '페인팅', '제공', '주식', '토크', '경제', '유투', '육아', '베이비', '특정', '자신', '특별', '매니아층', '토킹', '편하다', '과외', '영화', 

In [22]:
#Visualize the result

g = Network(width=800, height=800, font_color="#333")
correl = mdl.get_correlations().reshape([-1])

In [23]:
correl.sort()

In [24]:
# 상관계수 상위 10%만 간선으로 잇고
top_tenth = mdl.k * (mdl.k - 1) // 10
top_tenth = correl[-mdl.k - top_tenth]

In [25]:
topic_counts = mdl.get_count_by_topics()

In [26]:
print(topic_counts)

#합하면 242 (242words) >> 빈도수 242

[41 31 60 44 42 36 58 40 52 61]


In [27]:
for k in range(mdl.k):
    label = "#{}".format(k)
    title= ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=8))
    print('Topic', label, title)
    label += '\n' + ' '.join(word for word, _ in mdl.get_topic_words(k, top_n=3))
    g.add_node(k, label=label, title=title, shape='ellipse', value=float(topic_counts[k]))
    for l, correlation in zip(range(k - 1), mdl.get_correlations(k)):
        if correlation < top_tenth: continue
        g.add_edge(k, l, value=float(correlation), title='{:.02}'.format(correlation))

Topic #0 브이로그 주제 시청자 즐거움 로또 개인 과외 흥미
Topic #1 재미있다 콘텐츠 자연 연주 촬영 노래 요리 환경
Topic #2 소통 편집 자유 강아지 많이 귀엽다 실시간 나오다
Topic #3 일상 정보 공유 음악 업로드 제공 부르다 언덕
Topic #4 영상 패션 추억 운동 관련 고양이 먹방 직접
Topic #5 게임 취미 방송 리뷰 사용 아직 힘들다 최대한
Topic #6 영상 좋아하다 만들다 가장 보이다 어리다 영상미 즐기다
Topic #7 활동 교육 생활 공감 반려 직장 위하다 제품
Topic #8 여행 내용 분야 대하다 관심 가족 지식 전달
Topic #9 사람 라이브 모습 제작 처음 도움 기록 올리다


In [28]:
#토픽 topic_id에 속하는 상위 top_n개의 단어와 각각의 확률을 반환.
#반환 타입은 (단어:str, 확률:float) 튜플의 list형
#확률은 현재 토픽 내 각각의 단어들의 발생확률?

topic_words = mdl.get_topic_words(k, top_n=50)

In [29]:
import pandas as pd

In [30]:
df_topic_words = pd.DataFrame(topic_words)

In [31]:
#https://dandyrilla.github.io/2017-08-12/pandas-10min/

df_topic_words

,0,1
0,사람,0.068867
1,라이브,0.053354
2,모습,0.049614
3,제작,0.041380
4,처음,0.039143
5,도움,0.034997
6,기록,0.034395
7,올리다,0.029388
8,노력,0.027962
9,기존,0.026288


In [32]:
df_topic_words.describe()

,1
count,50.000000
mean,0.019822
std,0.014042
min,0.000040
25%,0.015491
50%,0.018157
75%,0.021819
max,0.068867


In [33]:
topic_words

[('사람', 0.06886745244264603),
 ('라이브', 0.053353872150182724),
 ('모습', 0.04961421713232994),
 ('제작', 0.041379958391189575),
 ('처음', 0.03914317861199379),
 ('도움', 0.03499670699238777),
 ('기록', 0.034394871443510056),
 ('올리다', 0.029387589544057846),
 ('노력', 0.027961965650320053),
 ('기존', 0.026288384571671486),
 ('편안', 0.024589626118540764),
 ('전달력', 0.024589626118540764),
 ('다른', 0.02181912399828434),
 ('육아', 0.02181912399828434),
 ('다시', 0.02181912399828434),
 ('메추리', 0.02181912399828434),
 ('베이비', 0.02181912399828434),
 ('인천', 0.02181912399828434),
 ('동네', 0.02181912399828434),
 ('밴드', 0.020198484882712364),
 ('홍보', 0.020198484882712364),
 ('관심거리', 0.019048621878027916),
 ('지금', 0.01815672032535076),
 ('그저', 0.01815672032535076),
 ('공략', 0.01815672032535076),
 ('기념', 0.01815672032535076),
 ('그대로', 0.01815672032535076),
 ('너무', 0.01815672032535076),
 ('의미', 0.01815672032535076),
 ('과제', 0.01742798276245594),
 ('학교', 0.01742798276245594),
 ('시연', 0.01742798276245594),
 ('시작', 0.01681184582

In [34]:
g.barnes_hut(gravity=-1000, spring_length=20)

In [35]:
g.show_buttons()
g.show("topic_network_210317_1.html")

In [36]:
import pyLDAvis
import pyLDAvis.gensim

/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/oridori/git/python/oridori/venv/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-no

In [37]:
pyLDAvis.enable_notebook()

In [38]:
topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])

In [41]:
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims= True)

In [44]:
prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency
    
)

In [45]:
pyLDAvis.display(prepared_data)

In [46]:
htmlFile = 'tomotopy_CTM TO LDA_210317.html'
pyLDAvis.save_html(prepared_data, htmlFile)

In [47]:
from bs4 import BeautifulSoup
with open(htmlFile) as inf:
    txt = inf.read()
    soup = BeautifulSoup(txt)

In [48]:
print(soup)

<html><head><link href="https://cdn.jsdelivr.net/gh/bmabey/pyLDAvis@3.2.2/pyLDAvis/js/ldavis.v1.0.0.css" rel="stylesheet" type="text/css"/>
</head><body><div id="ldavis_el17702140518197008864245933325"></div>
<script type="text/javascript">

var ldavis_el17702140518197008864245933325_data = {"mdsDat": {"x": [0.09459324801435896, 0.06542412495854247, -0.3211214422230044, 0.10502189098569704, 0.10235389089202643, 0.06087137482803627, 0.09194235265246133, -0.31861697732995653, 0.059713758180263116, 0.05981777904157534], "y": [0.23121538219137378, 0.024163429531272918, -0.010412025607054086, -0.26785193149049935, -0.2520476246337908, 0.022338503242676628, 0.21859804798791999, -0.009626503899107849, 0.021290597530670162, 0.022332125146538366], "topics": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], "cluster": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "Freq": [13.583990266541477, 12.913084102845762, 12.086494328963823, 11.376253471926844, 9.385779638012828, 9.124491878336379, 8.722164224892504, 8.543189120249895, 